# Scrape Upcoming Events

This script is dedicated to scraping and collecting upcoming Ultimate Fighting Championship (UFC) event and fighter data from the official UFC website (ufc.com) and UFC statistics site (ufcstats.com). Here is a summary of what each part of the code does:

1. The script starts with importing essential libraries like pandas, numpy, selenium, BeautifulSoup, etc.

2. The `get_next_events` function is designed to scrape upcoming events' data from the UFC website. It collects the title, URL, and date for each event.

3. The `get_event_fights` function gathers information about each fight happening in an event. It extracts details such as fighters' names, weight class, and odds.

4. The `secret_number` function uses Selenium and BeautifulSoup to automate browser actions to navigate the website and find a particular number, which seems to be specific to each event's matchup.

5. The `get_next_events2` function is similar to `get_next_events`, but it also removes events that lack fight data.

6. The `get_next_event_ufcstats` function scrapes data about upcoming events from the UFC Stats website.

7. The `get_fighter_urls` function collects fighter-related data such as name, URL of the profile from the UFC Stats website.

8. The `find_all_iframe_sources` function finds the source URLs for all iframes on a given UFC event page.

9. The `get_iframe_src_data_2` function uses the previously found iframe source URLs to gather detailed statistics about each fighter.

10. Each function gets applied iteratively to events and matchups, storing its collected information in pandas DataFrames for convenient manipulation and analysis.

11. Finally, the program saves the collated and cleaned data to a CSV file for further use.


In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from cmath import nan
import urllib
import urllib.request
import re
import time


In [2]:
def get_next_events(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('div', class_='c-card-event--result__info')
    event1_txt = soup.find('div', class_='c-card-event--result__info').text
    event1_url = event1.find('a')['href']
    event1_url = 'https://www.ufc.com' + event1_url
    event1_title = event1_txt.split('\n')[1]
    event1_time = event1_txt.split('/')[1]

    data = pd.DataFrame({'event_title': [event1_title], 'event_url': [event1_url], 'event_date': [event1_time]})

    event2 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event2_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event2_url = event2.find('a')['href']
    event2_url = 'https://www.ufc.com' + event2_url
    event2_title = event2_txt.split('\n')[1]
    event2_time = event2_txt.split('/')[1]


    data = data.append({'event_title': event2_title, 'event_url': event2_url, 'event_date': event2_time}, ignore_index=True)
    
    event3 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event3_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event3_url = event3.find('a')['href']
    event3_url = 'https://www.ufc.com' + event3_url
    event3_title = event3_txt.split('\n')[1]
    event3_time = event3_txt.split('/')[1]

    data = data.append({'event_title': event3_title, 'event_url': event3_url, 'event_date': event3_time}, ignore_index=True)
    
    return data

# Function to get the fight card for a given event using BeautifulSoup
def get_event_fights(event_url):
    page = requests.get(event_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get main card, fight 1

    mcn = soup.find_all('li', class_='l-listing__item')
    # get num of mc
    num_mc = len(mcn)
    # for each mc, do the following
    data = pd.DataFrame()
    n = 0
    for i in mcn:
        mc = mcn[n]
        # fight 1
        fighter1= mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--red').text
        fighter1 = fighter1.replace('\n', ' ')
        fighter1 = fighter1.strip()
        fighter2 = mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--blue').text
        fighter2 = fighter2.replace('\n', ' ')
        fighter2 = fighter2.strip()
        weightclass = mc.find('div', class_='c-listing-fight__class-text').text
        fighter1_odds = mc.find('span', class_='c-listing-fight__odds').text
        fighter2_odds = mc.find('span', class_='c-listing-fight__odds').find_next('span', class_='c-listing-fight__odds').text
        fighter1_odds = fighter1_odds.replace('\n', '')
        fighter2_odds = fighter2_odds.replace('\n', '')
        # fighter odds to float
        if (fighter1_odds == '-') :
            fighter1_odds = nan
        if (fighter2_odds == '-') :
            fighter2_odds = nan

        data = data.append({'fighter1': fighter1, 'fighter2': fighter2, 'weightclass': weightclass, 
                            'fighter1_odds': fighter1_odds, 'fighter2_odds': fighter2_odds}, ignore_index=True)
        n = n + 1
    return data

# Find secret number in ufc events using BS & Selenium
def secret_number(event_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\UFC_Prediction_V2\\chromedriver.exe')
    else:
        driver = driver
    
    driver.get(event_url)
    time.sleep(3)
    # click the first matchup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()
    # find first data-fmid to get first matchup
    fmid_start = pretty.find('data-fmid')
    fmid = pretty[fmid_start+11:fmid_start+16]
    driver.get(event_url +'#' + fmid)
    time.sleep(6)
    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')
    # find all links
    iframe_text = str(iframe)
    matchup = iframe_text.find('matchup')
    matchup_url = iframe_text[matchup+8:matchup+12]
    print('matchup_url: ' + matchup_url)
    secret_number = matchup_url
    return matchup

# get next events if event fighter data is not na
def get_next_events2(url):
    data = get_next_events(url)
    for i in range(0, len(data)):
        event_url = data['event_url'][i]
        event_fights = get_event_fights(event_url)
        if (len(event_fights) == 0):
            data = data.drop(i)
    return data

# get next events from UFCStats.com using BS
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data


# get fighter urls from UFCStats.com using BS
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data



In [3]:
next_events = get_next_events2('https://www.ufc.com/events')
next_events

,event_title,event_url,event_date
0,Almeida vs Lewis,https://www.ufc.com/event/ufc-fight-night-nove...,9:00 PM EDT
1,Jones vs Miocic,https://www.ufc.com/event/ufc-295,5:00 PM EST
2,Allen vs Craig,https://www.ufc.com/event/ufc-fight-night-nove...,5:00 PM EST


In [4]:
event_url =  next_events['event_url'][0]

In [5]:
next_event_title = next_events['event_title'][0]

In [6]:
next_event = get_event_fights(event_url)

In [7]:
page = requests.get(event_url)
soup = BeautifulSoup(page.content, 'html.parser')
h = soup.find_all('div', class_='c-listing-fight')

data_fmid = []
for i in h:
    data_fmid.append(i['data-fmid'])

next_event['fight_number'] = data_fmid[:len(next_event)]
next_event['matchup_url'] = event_url +'#' + next_event['fight_number'].astype(str)

next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url
0,Jailton Almeida,Derrick Lewis,Heavyweight Bout,-520,+390,10994,https://www.ufc.com/event/ufc-fight-night-nove...
1,Gabriel Bonfim,Nicolas Dalby,Welterweight Bout,-380,+300,10872,https://www.ufc.com/event/ufc-fight-night-nove...
2,Rodrigo Nascimento,Dontale Mayes,Heavyweight Bout,-218,+180,10867,https://www.ufc.com/event/ufc-fight-night-nove...
3,Caio Borralho,Abus Magomedov,Middleweight Bout,NaN,NaN,10997,https://www.ufc.com/event/ufc-fight-night-nove...
4,Rodolfo Vieira,Armen Petrosyan,Middleweight Bout,-102,-118,10870,https://www.ufc.com/event/ufc-fight-night-nove...
5,Ismael Bonfim,Vinc Pichel,Lightweight Bout,NaN,NaN,10952,https://www.ufc.com/event/ufc-fight-night-nove...
6,Elves Brener,Esteban Ribovics,Lightweight Bout,NaN,NaN,10864,https://www.ufc.com/event/ufc-fight-night-nove...
7,Victor Hugo,Daniel Marcos,Bantamweight Bout,NaN,NaN,11001,https://www.ufc.com/event/ufc-fight-night-nove...
8,Elizeu Zaleski dos Santos,Rinat Fakhretdinov,Welterweight Bout,NaN,NaN,10865,https://www.ufc.com/event/ufc-fight-night-nove...
9,Vitor Petrino,Modestas Bukauskas,Light Heavyweight Bout,NaN,NaN,10868,https://www.ufc.com/event/ufc-fight-night-nove...


In [8]:
def find_all_iframe_sources(matchup_url):
    try:
        driver = webdriver.Chrome('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/V2_Newer_Notebooks/chromedriver')
        driver.get(matchup_url)
        time.sleep(3)
        # get innerhtml
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        pretty = soup.prettify()
        # find all links within page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # find all iframe src
        iframe = soup.find_all('iframe')
        # list all links
        iframe_text = str(iframe)
        # separate links
        iframe_text = iframe_text.split('src="')
        iframe_text = iframe_text[1:]
        iframe_text = [i.split('"')[0] for i in iframe_text]
        # only keep links that contain matchup
        iframe_text = [i for i in iframe_text if 'matchup' in i]
        # only keep top link
        iframe_text = iframe_text[0]
        
        return iframe_text
    except:
        return 'error'


In [9]:
find_all_iframe_sources(next_event['matchup_url'][1])

'/matchup/1171/10872/pre'

Add iframe Sources to next_event. This can take a few minutes. 

In [10]:
# add iframe sources to df  with itterrows and apply
next_event['iframe_src'] = next_event.apply(lambda x: find_all_iframe_sources(x['matchup_url']), axis=1)

In [11]:
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src
0,Jailton Almeida,Derrick Lewis,Heavyweight Bout,-520,+390,10994,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10994/pre
1,Gabriel Bonfim,Nicolas Dalby,Welterweight Bout,-380,+300,10872,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10872/pre
2,Rodrigo Nascimento,Dontale Mayes,Heavyweight Bout,-218,+180,10867,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10867/pre
3,Caio Borralho,Abus Magomedov,Middleweight Bout,NaN,NaN,10997,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10997/pre
4,Rodolfo Vieira,Armen Petrosyan,Middleweight Bout,-102,-118,10870,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10870/pre
5,Ismael Bonfim,Vinc Pichel,Lightweight Bout,NaN,NaN,10952,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10952/pre
6,Elves Brener,Esteban Ribovics,Lightweight Bout,NaN,NaN,10864,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10864/pre
7,Victor Hugo,Daniel Marcos,Bantamweight Bout,NaN,NaN,11001,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/11001/pre
8,Elizeu Zaleski dos Santos,Rinat Fakhretdinov,Welterweight Bout,NaN,NaN,10865,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10865/pre
9,Vitor Petrino,Modestas Bukauskas,Light Heavyweight Bout,NaN,NaN,10868,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10868/pre


In [12]:
next_event['full_iframe_src'] = 'https://www.ufc.com' + next_event['iframe_src']

In [13]:
# iframe_response = requests.get('https://www.ufc.com/matchup/1162/10862/pre')
# iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
# red = iframe_soup.find_all('div', class_='c-stat-compare__group-1 red')
# red_text = [i.text for i in red]
# red_text

In [14]:
def get_iframe_src_data_2(iframe_src):
    iframe_response = requests.get(iframe_src)
    iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
    # get all c-stat-compare__group-1 red
    red = iframe_soup.find_all('div', class_='c-stat-compare__group-1 red')
    # return all red texts
    red_text = [i.text for i in red]
    # assign to variables
    red_record = red_text[0]
    red_last_fight = red_text[1]
    red_country = red_text[2]
    red_height = red_text[3]
    red_weight = red_text[4]
    red_reach = red_text[5]
    red_legreach = red_text[6]
    red_win_by_ko_percent = red_text[7]
    red_win_by_sub_percent = red_text[8]
    red_win_by_dec_percent = red_text[9]
    red_avg_fight_time = red_text[10]
    red_knockdowns_per_15_min = red_text[11]
    # sig strikes
    red_sig_strikes_landed_per_min = red_text[12]
    red_sig_strikes_percent = red_text[13]
    red_sig_strikes_absorbed_per_min = red_text[14]
    red_sig_strikes_absorbed_percent = red_text[15]
    # grappling
    red_takedowns_landed_per_15_min = red_text[16]
    red_takedown_accuracy = red_text[17]
    red_takedown_defense = red_text[18]
    red_submissions_attempts_per_15_min = red_text[19]
    # odds
    red_moneyline = red_text[20]


    # make df for red
    red_df = pd.DataFrame({'red_record': red_record, 'red_last_fight': red_last_fight, 
                            'red_country': red_country, 'red_height': red_height, 'red_weight': red_weight, 
                            'red_reach': red_reach, 'red_legreach': red_legreach, 
                            'red_win_by_ko_percent': red_win_by_ko_percent, 
                            'red_win_by_sub_percent': red_win_by_sub_percent, 'red_win_by_dec_percent': red_win_by_dec_percent, 
                            'red_avg_fight_time': red_avg_fight_time, 'red_knockdowns_per_15_min': red_knockdowns_per_15_min, 
                            'red_sig_strikes_landed_per_min': red_sig_strikes_landed_per_min, 
                            'red_sig_strikes_percent': red_sig_strikes_percent, 
                            'red_sig_strikes_absorbed_per_min': red_sig_strikes_absorbed_per_min, 
                            'red_sig_strikes_absorbed_percent': red_sig_strikes_absorbed_percent, 
                            'red_takedowns_landed_per_15_min': red_takedowns_landed_per_15_min, 
                            'red_takedown_accuracy': red_takedown_accuracy, 'red_takedown_defense': red_takedown_defense, 
                            'red_submissions_attempts_per_15_min': red_submissions_attempts_per_15_min, 
                            'red_moneyline': red_moneyline}, index=[0])

    # clean all values in red_df, removing all \n 
    red_df = red_df.applymap(lambda x: x.replace('\n', ''))
    # strip all values in red_df
    red_df = red_df.applymap(lambda x: x.strip())

    # get all c-stat-compare__group-1 blue
    blue = iframe_soup.find_all('div', class_='c-stat-compare__group-2 blue')
    # return all blue texts
    blue_text = [i.text for i in blue]
    # assign to variables
    blue_record = blue_text[0]
    blue_last_fight = blue_text[1]
    blue_country = blue_text[2]
    blue_height = blue_text[3]
    blue_weight = blue_text[4]
    blue_reach = blue_text[5]
    blue_legreach = blue_text[6]
    blue_win_by_ko_percent = blue_text[7]
    blue_win_by_sub_percent = blue_text[8]
    blue_win_by_dec_percent = blue_text[9]
    blue_avg_fight_time = blue_text[10]
    blue_knockdowns_per_15_min = blue_text[11]
    # sig strikes
    blue_sig_strikes_landed_per_min = blue_text[12]
    blue_sig_strikes_percent = blue_text[13]
    blue_sig_strikes_absorbed_per_min = blue_text[14]
    blue_sig_strikes_absorbed_percent = blue_text[15]
    # grappling
    blue_takedowns_landed_per_15_min = blue_text[16]
    blue_takedown_accuracy = blue_text[17]
    blue_takedown_defense = blue_text[18]
    blue_submissions_attempts_per_15_min = blue_text[19]
    # odds
    blue_moneyline = blue_text[20]


    # make df for blue
    blue_df = pd.DataFrame({'blue_record': blue_record, 'blue_last_fight': blue_last_fight,
                            'blue_country': blue_country, 'blue_height': blue_height, 'blue_weight': blue_weight,
                            'blue_reach': blue_reach, 'blue_legreach': blue_legreach,
                            'blue_win_by_ko_percent': blue_win_by_ko_percent,
                            'blue_win_by_sub_percent': blue_win_by_sub_percent, 'blue_win_by_dec_percent': blue_win_by_dec_percent,
                            'blue_avg_fight_time': blue_avg_fight_time, 'blue_knockdowns_per_15_min': blue_knockdowns_per_15_min,
                            'blue_sig_strikes_landed_per_min': blue_sig_strikes_landed_per_min,
                            'blue_sig_strikes_percent': blue_sig_strikes_percent,
                            'blue_sig_strikes_absorbed_per_min': blue_sig_strikes_absorbed_per_min,
                            'blue_sig_strikes_absorbed_percent': blue_sig_strikes_absorbed_percent,
                            'blue_takedowns_landed_per_15_min': blue_takedowns_landed_per_15_min,
                            'blue_takedown_accuracy': blue_takedown_accuracy, 'blue_takedown_defense': blue_takedown_defense,
                            'blue_submissions_attempts_per_15_min': blue_submissions_attempts_per_15_min,
                            'blue_moneyline': blue_moneyline}, index=[0])

    # clean all values in blue_df, removing all \n
    blue_df = blue_df.applymap(lambda x: x.replace('\n', ''))
    # strip all values in blue_df
    blue_df = blue_df.applymap(lambda x: x.strip())


    # append blue_df to red_df by axis=1
    dfs = pd.concat([red_df, blue_df], axis=1)

    # return df
    return dfs





In [15]:
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src,full_iframe_src
0,Jailton Almeida,Derrick Lewis,Heavyweight Bout,-520,+390,10994,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10994/pre,https://www.ufc.com/matchup/1171/10994/pre
1,Gabriel Bonfim,Nicolas Dalby,Welterweight Bout,-380,+300,10872,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10872/pre,https://www.ufc.com/matchup/1171/10872/pre
2,Rodrigo Nascimento,Dontale Mayes,Heavyweight Bout,-218,+180,10867,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10867/pre,https://www.ufc.com/matchup/1171/10867/pre
3,Caio Borralho,Abus Magomedov,Middleweight Bout,NaN,NaN,10997,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10997/pre,https://www.ufc.com/matchup/1171/10997/pre
4,Rodolfo Vieira,Armen Petrosyan,Middleweight Bout,-102,-118,10870,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10870/pre,https://www.ufc.com/matchup/1171/10870/pre
5,Ismael Bonfim,Vinc Pichel,Lightweight Bout,NaN,NaN,10952,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10952/pre,https://www.ufc.com/matchup/1171/10952/pre
6,Elves Brener,Esteban Ribovics,Lightweight Bout,NaN,NaN,10864,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10864/pre,https://www.ufc.com/matchup/1171/10864/pre
7,Victor Hugo,Daniel Marcos,Bantamweight Bout,NaN,NaN,11001,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/11001/pre,https://www.ufc.com/matchup/1171/11001/pre
8,Elizeu Zaleski dos Santos,Rinat Fakhretdinov,Welterweight Bout,NaN,NaN,10865,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10865/pre,https://www.ufc.com/matchup/1171/10865/pre
9,Vitor Petrino,Modestas Bukauskas,Light Heavyweight Bout,NaN,NaN,10868,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10868/pre,https://www.ufc.com/matchup/1171/10868/pre


In [16]:
rand_iframe_src = next_event['full_iframe_src'][0]
rand_iframe_src

'https://www.ufc.com/matchup/1171/10994/pre'

In [17]:
# get one iframe src data
get_iframe_src_data_2(rand_iframe_src)

,red_record,red_last_fight,red_country,red_height,red_weight,red_reach,red_legreach,red_win_by_ko_percent,red_win_by_sub_percent,red_win_by_dec_percent,...,blue_knockdowns_per_15_min,blue_sig_strikes_landed_per_min,blue_sig_strikes_percent,blue_sig_strikes_absorbed_per_min,blue_sig_strikes_absorbed_percent,blue_takedowns_landed_per_15_min,blue_takedown_accuracy,blue_takedown_defense,blue_submissions_attempts_per_15_min,blue_moneyline
0,19-2-0,Win,Brazil,"6' 3""",231.00 LB,79 in,43.5 in,37%,63%,0%,...,0.59,2.6,50%,2.59,41%,0.59,28%,52%,0,+390


In [18]:
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src,full_iframe_src
0,Jailton Almeida,Derrick Lewis,Heavyweight Bout,-520,+390,10994,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10994/pre,https://www.ufc.com/matchup/1171/10994/pre
1,Gabriel Bonfim,Nicolas Dalby,Welterweight Bout,-380,+300,10872,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10872/pre,https://www.ufc.com/matchup/1171/10872/pre
2,Rodrigo Nascimento,Dontale Mayes,Heavyweight Bout,-218,+180,10867,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10867/pre,https://www.ufc.com/matchup/1171/10867/pre
3,Caio Borralho,Abus Magomedov,Middleweight Bout,NaN,NaN,10997,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10997/pre,https://www.ufc.com/matchup/1171/10997/pre
4,Rodolfo Vieira,Armen Petrosyan,Middleweight Bout,-102,-118,10870,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10870/pre,https://www.ufc.com/matchup/1171/10870/pre
5,Ismael Bonfim,Vinc Pichel,Lightweight Bout,NaN,NaN,10952,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10952/pre,https://www.ufc.com/matchup/1171/10952/pre
6,Elves Brener,Esteban Ribovics,Lightweight Bout,NaN,NaN,10864,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10864/pre,https://www.ufc.com/matchup/1171/10864/pre
7,Victor Hugo,Daniel Marcos,Bantamweight Bout,NaN,NaN,11001,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/11001/pre,https://www.ufc.com/matchup/1171/11001/pre
8,Elizeu Zaleski dos Santos,Rinat Fakhretdinov,Welterweight Bout,NaN,NaN,10865,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10865/pre,https://www.ufc.com/matchup/1171/10865/pre
9,Vitor Petrino,Modestas Bukauskas,Light Heavyweight Bout,NaN,NaN,10868,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10868/pre,https://www.ufc.com/matchup/1171/10868/pre


In [19]:
# get iframe src data for all events

event_data = []

for i in range(len(next_event['full_iframe_src'])):
    try:
        event_data.append(get_iframe_src_data_2(next_event['full_iframe_src'][i]))
    except:
        event_data.append(pd.DataFrame([{'error': 'error'}]))

# make df from event_data
event_data_df = pd.concat(event_data, axis=0)

event_data_df

,red_record,red_last_fight,red_country,red_height,red_weight,red_reach,red_legreach,red_win_by_ko_percent,red_win_by_sub_percent,red_win_by_dec_percent,...,blue_knockdowns_per_15_min,blue_sig_strikes_landed_per_min,blue_sig_strikes_percent,blue_sig_strikes_absorbed_per_min,blue_sig_strikes_absorbed_percent,blue_takedowns_landed_per_15_min,blue_takedown_accuracy,blue_takedown_defense,blue_submissions_attempts_per_15_min,blue_moneyline
0,19-2-0,Win,Brazil,"6' 3""",231.00 LB,79 in,43.5 in,37%,63%,0%,...,0.59,2.6,50%,2.59,41%,0.59,28%,52%,0,+390
0,15-0-0,Win,Brazil,"6' 1""",171.00 LB,72.5 in,40.5 in,20%,80%,0%,...,0,3.93,43%,3.39,52%,1.18,31%,62%,0.1,+300
0,"10-1-0, 2NC",Win,Brazil,"6' 2""",264.50 LB,80 in,43 in,22%,67%,11%,...,0.26,3.39,43%,3.73,44%,0.89,46%,56%,0,+180
0,"14-1-0, 1NC",Win,Brazil,"5' 10""",185.50 LB,75 in,42 in,31%,23%,46%,...,1.55,5.6,50%,8.39,57%,1.55,50%,0%,0,N/A
0,9-2-0,Win,Brazil,"6' 0""",186.00 LB,73 in,39.5 in,11%,89%,0%,...,0.23,5.97,52%,3.03,56%,0.23,33%,36%,0,-118
0,19-4-0,Loss,Brazil,"5' 8""",156.00 LB,71.5 in,35 in,47%,21%,32%,...,0.13,3.56,47%,2.5,58%,3,54%,23%,0.25,N/A
0,15-3-0,Win,Brazil,"5' 10""",155.00 LB,72.5 in,39.5 in,7%,79%,14%,...,0.95,6.16,52%,3.4,52%,0.95,66%,60%,0.48,N/A
0,12-2-0,Debut,"0' 0""",LB,0 in,0 in,0%,0%,0%,:00,...,4.88,46%,3.7,69%,0.4,50%,89%,0,N/A,N/A
0,24-7-0,Win,Brazil,"5' 11""",171.00 LB,73 in,41 in,58%,13%,29%,...,0.97,3.2,67%,0.78,57%,5.82,63%,100%,0.49,N/A
0,9-0-0,Win,Brazil,"6' 2""",206.00 LB,77.5 in,42.5 in,86%,0%,14%,...,0,3.46,40%,4.13,48%,0,0%,75%,0,N/A


In [20]:
# reindex both
next_event = next_event.reset_index(drop=True)
event_data_df = event_data_df.reset_index(drop=True)

# Append event_data_df to next_event
next_event = pd.concat([next_event, event_data_df], axis=1)
next_event

,fighter1,fighter2,weightclass,fighter1_odds,fighter2_odds,fight_number,matchup_url,iframe_src,full_iframe_src,red_record,...,blue_knockdowns_per_15_min,blue_sig_strikes_landed_per_min,blue_sig_strikes_percent,blue_sig_strikes_absorbed_per_min,blue_sig_strikes_absorbed_percent,blue_takedowns_landed_per_15_min,blue_takedown_accuracy,blue_takedown_defense,blue_submissions_attempts_per_15_min,blue_moneyline
0,Jailton Almeida,Derrick Lewis,Heavyweight Bout,-520,+390,10994,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10994/pre,https://www.ufc.com/matchup/1171/10994/pre,19-2-0,...,0.59,2.6,50%,2.59,41%,0.59,28%,52%,0,+390
1,Gabriel Bonfim,Nicolas Dalby,Welterweight Bout,-380,+300,10872,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10872/pre,https://www.ufc.com/matchup/1171/10872/pre,15-0-0,...,0,3.93,43%,3.39,52%,1.18,31%,62%,0.1,+300
2,Rodrigo Nascimento,Dontale Mayes,Heavyweight Bout,-218,+180,10867,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10867/pre,https://www.ufc.com/matchup/1171/10867/pre,"10-1-0, 2NC",...,0.26,3.39,43%,3.73,44%,0.89,46%,56%,0,+180
3,Caio Borralho,Abus Magomedov,Middleweight Bout,NaN,NaN,10997,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10997/pre,https://www.ufc.com/matchup/1171/10997/pre,"14-1-0, 1NC",...,1.55,5.6,50%,8.39,57%,1.55,50%,0%,0,N/A
4,Rodolfo Vieira,Armen Petrosyan,Middleweight Bout,-102,-118,10870,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10870/pre,https://www.ufc.com/matchup/1171/10870/pre,9-2-0,...,0.23,5.97,52%,3.03,56%,0.23,33%,36%,0,-118
5,Ismael Bonfim,Vinc Pichel,Lightweight Bout,NaN,NaN,10952,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10952/pre,https://www.ufc.com/matchup/1171/10952/pre,19-4-0,...,0.13,3.56,47%,2.5,58%,3,54%,23%,0.25,N/A
6,Elves Brener,Esteban Ribovics,Lightweight Bout,NaN,NaN,10864,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10864/pre,https://www.ufc.com/matchup/1171/10864/pre,15-3-0,...,0.95,6.16,52%,3.4,52%,0.95,66%,60%,0.48,N/A
7,Victor Hugo,Daniel Marcos,Bantamweight Bout,NaN,NaN,11001,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/11001/pre,https://www.ufc.com/matchup/1171/11001/pre,12-2-0,...,4.88,46%,3.7,69%,0.4,50%,89%,0,N/A,N/A
8,Elizeu Zaleski dos Santos,Rinat Fakhretdinov,Welterweight Bout,NaN,NaN,10865,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10865/pre,https://www.ufc.com/matchup/1171/10865/pre,24-7-0,...,0.97,3.2,67%,0.78,57%,5.82,63%,100%,0.49,N/A
9,Vitor Petrino,Modestas Bukauskas,Light Heavyweight Bout,NaN,NaN,10868,https://www.ufc.com/event/ufc-fight-night-nove...,/matchup/1171/10868/pre,https://www.ufc.com/matchup/1171/10868/pre,9-0-0,...,0,3.46,40%,4.13,48%,0,0%,75%,0,N/A


In [21]:
# save to csv
next_event.to_csv('data/final/next_fights/'+ next_event_title +'_.csv', index=False)